# Live embryo analysis for saykalib

### 10/01/24
### Author: Andy D. Tran, CCR Microscopy Core, LCBG, CCR, NCI

In [1]:
import os 
import numpy as np
import pandas as pd 
import napari 
import re 

from scipy import ndimage as nd 
from aicsimageio import AICSImage 
from imaris_ims_file_reader.ims import ims
from tifffile import imread, imwrite 
from skimage.measure import label, regionprops
from skimage.segmentation import expand_labels
from tqdm import tqdm

### Open and prepare images

In [79]:
base_path = 'Z:/IMAGES/saykalib/2024/Nikon SoRa/09_13'

In [88]:
tmp_list = os.listdir(base_path)
img_list = [] 
for tmp in tmp_list:
    if re.search('mtg', tmp):
        img_list.append(tmp)
print(img_list)
print(len(img_list))

['240913_liDHB_live_day0_mtg.nd2', '240913_liDHB_live_day0_mtg001.nd2', '240913_liDHB_live_day0_mtg_M1.nd2', '240913_liDHB_live_day0_mtg_M2.nd2', '240913_liDHB_live_day0_mtg_M3.nd2', '240913_liDHB_live_day0_mtg_M4.nd2', '240913_liDHB_live_day0_mtg_M5.nd2']
7


In [110]:
img_name = img_list[6]
img_path = os.path.join(base_path, img_name)

img_tmp = AICSImage(img_path)
print(img_tmp.shape)
print(img_tmp.channel_names)

(78, 2, 32, 2200, 2200)
['488nm', '561nm']


In [111]:
img = img_tmp.data

In [112]:
img_ch1 = img[:, 0, :, :, :]
img_ch2 = img[:, 1, :, :, :]

In [113]:
viewer = napari.view_image(img_ch2)

In [114]:
print(img_ch2.shape)

(78, 32, 2200, 2200)


In [115]:
output_path = 'Z:/Analysis/[NCI] [LGI] Sergio Ruiz/Bechara Saykali/images/live_embryo/live/Nikon_09_13/position_5'

In [116]:
time_num = img_ch2.shape[0]
for time_index in tqdm(range(time_num)):
    img_slice = img_ch2[time_index, :, :, :]
    slice_name = 'h2b_t' + str(time_index) + '.tif'
    slice_path = os.path.join(output_path, slice_name)
    imwrite(slice_path, img_slice)

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [03:52<00:00,  2.98s/it]


### Fixed images

In [8]:
base_path = 'Z:/IMAGES/saykalib/2024/ZeissLSM880/09_18'
output_path = 'Z:/Analysis/[NCI] [LGI] Sergio Ruiz/Bechara Saykali/images/live_embryo/fixed/09_18'

In [9]:
tmp_list = os.listdir(base_path)
img_list = [] 
for tmp in tmp_list:
    if re.search('IDHB', tmp):
        img_list.append(tmp)
print(img_list)
print(len(img_list))

['240918_240908_IDHB_cdx2_sox2_2024_09_18__17_10_23.czi', '240918_240908_IDHB_cdx2_sox2_2_2024_09_18__17_16_09.czi', '240918_240908_IDHB_cdx2_sox2_3_2024_09_18__17_19_34.czi', '240918_240908_IDHB_cdx2_sox2_4_2024_09_18__17_24_18.czi', '240918_240912_IDHB_cdx2_sox2_1_2024_09_18__17_28_48.czi', '240918_240912_IDHB_cdx2_sox2_2_2024_09_18__17_32_56.czi', '240918_240912_IDHB_cdx2_sox2_3_2024_09_18__17_36_25.czi', '240918_240912_IDHB_cdx2_sox2_4_2024_09_18__17_41_05.czi', '240918_240912_IDHB_cdx2_sox2_5_2024_09_18__17_44_28.czi', '240918_240913_IDHB_cdx2_sox2_1_2024_09_18__17_51_05.czi', '240918_240913_IDHB_cdx2_sox2_2_2024_09_18__17_54_53.czi', '240918_240913_IDHB_cdx2_sox2_3_2024_09_18__18_00_27.czi', '240918_240913_IDHB_cdx2_sox2_4_2024_09_18__18_05_56.czi', '240918_240913_IDHB_cdx2_sox2_5_2024_09_18__18_09_58.czi']
14


In [74]:
img_name = img_list[13]
img_path = os.path.join(base_path, img_name)

img_tmp = AICSImage(img_path)
print(img_tmp.shape)
print(img_tmp.channel_names)

(1, 4, 25, 1024, 1024)
['Ch1-T1', 'ChS1-T2', 'ChS2-T3', 'Ch2-T4']


In [75]:
img = img_tmp.data

img_ch1 = img[:, 0, :, :, :]
img_ch2 = img[:, 1, :, :, :]
img_ch3 = img[:, 2, :, :, :]
img_ch4 = img[:, 3, :, :, :]

In [76]:
viewer = napari.view_image(img_ch3)

In [77]:
viewer.close()
print(img_ch3.shape)

(1, 25, 1024, 1024)


In [78]:
nuc_path = os.path.join(output_path, img_name.replace('.czi', '_nuc.tif'))
imwrite(nuc_path, img_ch3)

### Stitch together live masks

In [19]:
base_path = 'Z:/Analysis/[NCI] [LGI] Sergio Ruiz/Bechara Saykali/images/live_embryo/live/09_12/position_5'

In [20]:
tmp_list = os.listdir(base_path)
img_list = []
for tmp in tmp_list:
    if re.search('_cp_masks.tif', tmp):
        img_list.append(tmp)

print(img_list)
print(len(img_list))

['h2b_t0_cp_masks.tif', 'h2b_t10_cp_masks.tif', 'h2b_t11_cp_masks.tif', 'h2b_t12_cp_masks.tif', 'h2b_t13_cp_masks.tif', 'h2b_t14_cp_masks.tif', 'h2b_t15_cp_masks.tif', 'h2b_t16_cp_masks.tif', 'h2b_t17_cp_masks.tif', 'h2b_t18_cp_masks.tif', 'h2b_t19_cp_masks.tif', 'h2b_t1_cp_masks.tif', 'h2b_t20_cp_masks.tif', 'h2b_t21_cp_masks.tif', 'h2b_t22_cp_masks.tif', 'h2b_t23_cp_masks.tif', 'h2b_t24_cp_masks.tif', 'h2b_t25_cp_masks.tif', 'h2b_t26_cp_masks.tif', 'h2b_t27_cp_masks.tif', 'h2b_t28_cp_masks.tif', 'h2b_t29_cp_masks.tif', 'h2b_t2_cp_masks.tif', 'h2b_t30_cp_masks.tif', 'h2b_t31_cp_masks.tif', 'h2b_t32_cp_masks.tif', 'h2b_t33_cp_masks.tif', 'h2b_t34_cp_masks.tif', 'h2b_t35_cp_masks.tif', 'h2b_t36_cp_masks.tif', 'h2b_t37_cp_masks.tif', 'h2b_t38_cp_masks.tif', 'h2b_t39_cp_masks.tif', 'h2b_t3_cp_masks.tif', 'h2b_t40_cp_masks.tif', 'h2b_t41_cp_masks.tif', 'h2b_t42_cp_masks.tif', 'h2b_t43_cp_masks.tif', 'h2b_t44_cp_masks.tif', 'h2b_t45_cp_masks.tif', 'h2b_t46_cp_masks.tif', 'h2b_t47_cp_masks.t

In [21]:
img_path = os.path.join(base_path, img_list[0])
img_tmp = imread(img_path)
print(img_tmp.shape)

(28, 1024, 1024)


In [22]:
mask_img = np.empty([0, 28, 1024, 1024])
for img_index in tqdm(range(len(img_list))):
    img_name = 'h2b_t' + str(img_index) + '_cp_masks.tif'
    img_path = os.path.join(base_path, img_name)
    img_tmp = imread(img_path)
    print(img_tmp.shape)
    img_tmp = np.expand_dims(img_tmp, axis = 0)
    mask_img = np.concatenate((mask_img, img_tmp), axis = 0)
print(mask_img.shape)

  2%|█▎                                                                                 | 1/64 [00:00<00:09,  6.80it/s]

(28, 1024, 1024)


  3%|██▌                                                                                | 2/64 [00:01<00:39,  1.56it/s]

(28, 1024, 1024)
(28, 1024, 1024)


  5%|███▉                                                                               | 3/64 [00:02<00:52,  1.16it/s]

...

(28, 1024, 1024)


 98%|████████████████████████████████████████████████████████████████████████████████▋ | 63/64 [04:21<00:07,  7.07s/it]

(28, 1024, 1024)


100%|██████████████████████████████████████████████████████████████████████████████████| 64/64 [04:29<00:00,  4.21s/it]

(64, 28, 1024, 1024)


In [23]:
viewer = napari.view_image(mask_img)

In [24]:
mask_path = os.path.join(base_path, 'h2b_masks.tif')
imwrite(mask_path, mask_img)

### Merge broken timelapses

In [17]:
tl1_path = 'Z:/Analysis/[NCI] [LGI] Sergio Ruiz/Bechara Saykali/images/live_embryo/live/09_12/240912_iDHB_live_day0_mtg_2024_09_12__15_39_18(4)/240912_iDHB_live_day0_mtg_2024_09_12__15_39_18(4).czi'
tl2_path = 'Z:/Analysis/[NCI] [LGI] Sergio Ruiz/Bechara Saykali/images/live_embryo/live/09_13/240913_iDHB_live_day1_mtg_2024_09_13__12_57_25(4)/240913_iDHB_live_day1_mtg_2024_09_13__12_57_25(4).czi'

In [18]:
tl1_tmp = AICSImage(tl1_path)
print(tl1_tmp.shape)

(64, 2, 28, 1024, 1024)


In [19]:
tl2_tmp = AICSImage(tl2_path)
print(tl2_tmp.shape)

(18, 2, 31, 1024, 1024)


In [20]:
tl1 = tl1_tmp.data
tl2 = tl2_tmp.data

In [21]:
tl_pad = np.zeros([64, 2, 1, 1024, 1024])
tl_pad = np.uint16(tl_pad)
tl_pad2 = np.zeros([64, 2, 2, 1024, 1024])
tl_pad2 = np.uint16(tl_pad2)

In [22]:
tl1 = np.concatenate((tl_pad, tl1), axis = 2)

In [23]:
tl1 = np.concatenate((tl1, tl_pad2), axis = 2)

In [24]:
tl_merge = np.concatenate((tl1, tl2), axis = 0)

In [25]:
tl_merge = np.uint16(tl_merge)

In [26]:
print(tl_merge.dtype)

uint16


In [27]:
print(tl_merge.shape)

(82, 2, 31, 1024, 1024)


In [28]:
output_path = 'Z:/Analysis/[NCI] [LGI] Sergio Ruiz/Bechara Saykali/images/live_embryo/live/09_13/240913_iDHB_live_day1_mtg_2024_09_13__12_57_25(4)/merge'

In [29]:
for time_index in tqdm(range(tl_merge.shape[0])):
    time_slice = tl_merge[time_index, :, :, :, :]
    for ch_index in range(time_slice.shape[0]):
        channel_slice = time_slice[ch_index, :, :, :]
        for z_index in range(channel_slice.shape[0]):
            z_slice = channel_slice[z_index, :, :]

            file_name = 'merge_' + str(time_index) + '_' + str(ch_index) + '_' + str(z_index) + '.tif'
            file_path = os.path.join(output_path, file_name)
            imwrite(file_path, z_slice)

100%|██████████████████████████████████████████████████████████████████████████████████| 82/82 [06:01<00:00,  4.41s/it]


### Cytoplasm extension

In [2]:
# Define functions------------------------------------------------------------------------------

def cyto_expand(img, px): 
    cyto_shell = expand_labels(img, distance = px) 
    cyto_buffer = expand_labels(img, distance = 2) 
    cyto = np.subtract(cyto_shell, cyto_buffer) 

    return cyto 

def roi_quant(label_img, dhb_img):
    df = pd.DataFrame() 

    nuc_id = []
    area = []
    intensity = []

    roi_props = regionprops(label_img, intensity_image = dhb_img)
    for roi in tqdm(range(len(roi_props))):
        nuc_id.append(roi_props[roi].label)
        area.append(roi_props[roi].area)
        intensity.append(roi_props[roi].mean_intensity)

    df['nuc_id'] = nuc_id
    df['area'] = area
    df['intensity'] = intensity

    return df

In [3]:
img_path = '/Volumes/LECIMAGE/Analysis/[NCI] [LGI] Sergio Ruiz/Bechara Saykali/images/live_embryo/live/09_12/240912_iDHB_live_day0_mtg_2024_09_12__15_39_18(2)/merge_0_0_0.ims'
output_path = '/Volumes/LECIMAGE/Analysis/[NCI] [LGI] Sergio Ruiz/Bechara Saykali/images/live_embryo/cyto_nuc_output/240912_iDHB_live_day0_mtg_2024_09_12__15_39_18(2)'

In [4]:
img_tmp = ims(img_path)
print(img_tmp.shape)
print(img_tmp.ndim)

Opening readonly file: /Volumes/LECIMAGE/Analysis/[NCI] [LGI] Sergio Ruiz/Bechara Saykali/images/live_embryo/live/09_12/240912_iDHB_live_day0_mtg_2024_09_12__15_39_18(2)/merge_0_0_0.ims 

(82, 3, 31, 1024, 1024)
5


In [5]:
dhb_img = img_tmp[:, 0, :, :, :]
#ruby_img = img_tmp[:, 1, :, :, :]
nuc_img = img_tmp[:, 2, :, :, :]

In [6]:
#viewer = napari.view_image(dhb_img)

Invalid schema for package 'ome-types', please run 'npe2 validate ome-types' to check for manifest errors.


In [7]:
for time_point in range(img_tmp.shape[0]):
    print('Time point: ' + str(time_point))
    dhb_time = dhb_img[time_point, :, :, :]
    nuc_time = nuc_img[time_point, :, :, :]
    for z_plane in range(img_tmp.shape[2]):
        print('Z plane: ' + str(z_plane))
        dhb_slice = dhb_time[z_plane, :, :]
        nuc_slice = nuc_time[z_plane, :, :]
        cyto_slice = cyto_expand(nuc_slice, 12)
        cyto_name = 'cyto_' + str(time_point) + '_' + str(z_plane) + '.tif'
        cyto_path = os.path.join(output_path, cyto_name)
        imwrite(cyto_path, cyto_slice)
        nuc_df = roi_quant(nuc_slice, dhb_slice)
        nuc_df['time_point'] = time_point
        nuc_df['z_plane'] = z_plane
        nuc_df_name = 'nuc_output_' + str(time_point) + '_' + str(z_plane) + '.csv'
        nuc_df_path = os.path.join(output_path, nuc_df_name)
        nuc_df.to_csv(nuc_df_path, header = True)
        cyto_df = roi_quant(cyto_slice, dhb_slice)
        cyto_df['time_point'] = time_point
        cyto_df['z_plane'] = z_plane
        cyto_df_name = 'cyto_output_' + str(time_point) + '_' + str(z_plane) + '.csv'
        cyto_df_path = os.path.join(output_path, cyto_df_name)
        cyto_df.to_csv(cyto_df_path, header = True)

Time point: 0
Z plane: 0


100%|███████████████████████████████████████████| 3/3 [00:00<00:00, 1311.68it/s]


Z plane: 1


100%|███████████████████████████████████████████| 3/3 [00:00<00:00, 1098.08it/s]


Z plane: 2


...

100%|█████████████████████████████████████████| 31/31 [00:00<00:00, 8857.78it/s]


Z plane: 30


100%|█████████████████████████████████████████| 24/24 [00:00<00:00, 8917.73it/s]


Z plane: 31


100%|████████████████████████████████████████| 21/21 [00:00<00:00, 11104.44it/s]
